In [2]:
import re
from thesisutils import utils
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

In [3]:
def find_entity(qrow, ner):
    # neridx = ner rows from article of quote in qrow
    neridx = ner[ner.Art_id.eq(qrow.Art_id)]
    # convert string tuple to py tuple
    tup = eval(qrow.speaker_index)
    overlaps = neridx[(neridx.start_char.ge(tup[0])) & (neridx.start_char.lt(tup[1]))]
    return overlaps[['entity', "ner_index"]].values.tolist()

In [4]:
def getdfs(pub):
    quotedf = utils.standardize(utils.read_df_s3(f"{pub.name}/quotes/quotes_full_edits.csv"), pub, drop_dups=False)
    nerdf = utils.standardize(utils.read_df_s3(f"{pub.name}/ner/ner_full2.csv"), pub, drop_dups=False)
    datedf = utils.standardize(utils.read_df_s3(f"{pub.name}/date/date.csv"), pub, drop_dups=False)
    quotedf = (
    quotedf.merge(datedf[["Art_id", "Year"]], on="Art_id")
    .drop_duplicates(subset=quotedf.columns.difference(quotedf.filter(regex="Unnamed", axis=1).columns))
    )
    nerdf = (
        nerdf.merge(datedf[["Art_id", "Year"]], on="Art_id")
        .drop_duplicates(subset=nerdf.columns.difference(['ner_index']))
    )
    ppl = nerdf[nerdf.label_.eq("PERSON")]
    return quotedf, nerdf, ppl, datedf

In [5]:
def concat2s3(qdfconcat, pub):
    qdfconcat = qdfconcat.drop(["Year", "Publication", "entities"], axis=1, errors="ignore")
    qdfconcat.index = qdfconcat.index.set_names(["quid"])
    qdfconcat = qdfconcat.reset_index()
    utils.df_to_s3(qdfconcat, f"{pub.name}/quotes/quotes_full_edits_new716.csv")
    return qdfconcat

In [ ]:
# pub = utils.publications['hkfp']
# quotedf, nerdf, ppl, _ = getdfs(pub)
# qdfls = []
# for year in range(2011, 2022):
#     # year = 2019
#     # make it a manageable size
#     qyr = quotedf[quotedf.Year.eq(year)]
#     nyr = ppl[ppl.Year.eq(year)]
#     if len(qyr) != 0 and len(nyr) != 0:
#         qyr['entities'] = qyr.progress_apply(find_entity, ner=nyr, axis=1)
#         x = qyr.entities.apply(pd.Series)
#         for col in x.columns:
#             for i, type_ in enumerate(("entity", "nerindex")):
#                 qyr['{}_{}'.format(type_, str(col+1))]=[k[i] if isinstance(k,list) else k for k in x[col]]
#         # convert to ints
#         for col in qyr.filter(regex="nerindex").columns:
#             if qyr[col].dtype == "float64":
#                 qyr[col] = qyr[col].astype(pd.Int64Dtype())#)
#         qdfls.append(qyr)
#     else:
#         print(f"only found {len(qyr)} quotes and {len(nyr)} entities")

# qdfconcat = pd.concat(qdfls)
# qdfconcat=concat2s3(qdfconcat, pub)

In [27]:
pub = utils.publications['globaltimes']
quotedf, nerdf, ppl, _ = getdfs(pub)

In [28]:
qdfls = []
for year in range(2011, 2022):
    # year = 2019
    # make it a manageable size
    qyr = quotedf[quotedf.Year.eq(year)]
    nyr = ppl[ppl.Year.eq(year)]
    if len(qyr) != 0 and len(nyr) != 0:
        qyr['entities'] = qyr.progress_apply(find_entity, ner=nyr, axis=1)
        x = qyr.entities.apply(pd.Series)
        for col in x.columns:
            for i, type_ in enumerate(("entity", "nerindex")):
                qyr['{}_{}'.format(type_, str(col+1))]=[k[i] if isinstance(k,list) else k for k in x[col]]
        # convert to ints
        for col in qyr.filter(regex="nerindex").columns:
            if qyr[col].dtype == "float64":
                qyr[col] = qyr[col].astype(pd.Int64Dtype())#)
        qdfls.append(qyr)
    else:
        print(f"only found {len(qyr)} quotes and {len(nyr)} entities")

qdfconcat = pd.concat(qdfls)

100%|██████████| 9815/9815 [00:13<00:00, 724.72it/s]
/tmp/ipykernel_14389/303452412.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qyr['entities'] = qyr.progress_apply(find_entity, ner=nyr, axis=1)
/tmp/ipykernel_14389/303452412.py:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  x = qyr.entities.apply(pd.Series)
/tmp/ipykernel_14389/303452412.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [29]:
qdfconcat=concat2s3(qdfconcat, pub)

In [31]:
pub = utils.publications['chinadaily']
quotedf, nerdf, ppl, _ = getdfs(pub)

/home/ec2-user/SageMaker/.persisted_conda/nlp/lib/python3.10/site-packages/thesisutils/utils.py:140: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(csv_string))
/home/ec2-user/SageMaker/.persisted_conda/nlp/lib/python3.10/site-packages/thesisutils/utils.py:140: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(csv_string))


In [5]:
qdfls = []
for year in range(2011, 2022):
    # year = 2019
    # make it a manageable size
    qyr = quotedf[quotedf.Year.eq(year)]
    nyr = ppl[ppl.Year.eq(year)]
    if len(qyr) != 0 and len(nyr) != 0:
        qyr['entities'] = qyr.progress_apply(find_entity, ner=nyr, axis=1)
        x = qyr.entities.apply(pd.Series)
        for col in x.columns:
            for i, type_ in enumerate(("entity", "nerindex")):
                qyr['{}_{}'.format(type_, str(col+1))]=[k[i] if isinstance(k,list) else k for k in x[col]]
        # convert to ints
        for col in qyr.filter(regex="nerindex").columns:
            if qyr[col].dtype == "float64":
                qyr[col] = qyr[col].astype(pd.Int64Dtype())#)
        qdfls.append(qyr)
    else:
        print(f"only found {len(qyr)} quotes and {len(nyr)} entities")

qdfconcat = pd.concat(qdfls)

NameError: name 'quotedf' is not defined

In [33]:
qdfconcat=concat2s3(qdfconcat, pub)

In [ ]:
qdfconcat.head()

,Unnamed: 0.1,Unnamed: 0,long_speaker,prepro_speaker,source,single_speaker,Publication
0,0,0,hamish tyrwhitt leighton asia managing director,hamish tyrwhitt leighton asia managing director,721978,False,scmp
1,1,1,NaN,tyrwhitt,721978,True,scmp
2,2,2,the young professionals club,he,721978,True,scmp
3,3,3,NaN,tyrwhitt,721978,True,scmp
4,4,4,the city commerce commission,the city commerce commission,721979,False,scmp
...,...,...,...,...,...,...,...
1730223,258053,258053,abuza of the national war college,abuza of the national war college,1203366,False,scmp
1730224,258054,258054,NaN,abuza,1203366,True,scmp
1730225,258055,258055,NaN,prayuth,1203366,True,scmp
1730226,258056,258056,NaN,chambers,1203366,True,scmp


In [7]:

# def getdfs(pub):

# nerdf = (
#     nerdf.merge(datedf[["Art_id", "Year"]], on="Art_id")
#     .drop_duplicates(subset=nerdf.columns.difference(['ner_index']))
# )
# ppl = nerdf[nerdf.label_.eq("PERSON")]
# return quotedf, nerdf, ppl, datedf



In [8]:
qdfls = []
pub = utils.publications['scmp']

for year in range(2011, 2021):
    # year = 2019
    # make it a manageable size
    quotedf = utils.standardize(utils.read_df_s3(f"{pub.name}/quotes/q_{year}.csv"), pub, drop_dups=False)
    quotedf["Art_id"] = quotedf.source # idk why there is this bug there...?
    quotedfedits = utils.read_df_s3(f"{pub.name}/quotes/q{year}_edits.csv")
    # quotedfedits["Art_id"] = quotedfedits.source # idk why there is this bug there...?
    quotedfedits = quotedfedits.drop("source", axis=1)

    quotedf = quotedf.drop("source", axis=1)
    # nerdf = utils.standardize(utils.read_df_s3(f"{pub.name}/ner/ner_full2.csv"), pub, drop_dups=False)
    # datedf = utils.standardize(utils.read_df_s3(f"{pub.name}/date/date.csv"), pub, drop_dups=False)
    quotedf = (
    quotedf
        #.merge(datedf[["Art_id", "Year"]], on="Art_id")
    .drop_duplicates(subset=quotedf.columns.difference(quotedf.filter(regex="Unnamed", axis=1).columns))
    )
    qyr = quotedf.merge(quotedfedits, left_index=True, right_index=True, how="left")#[quotedf.Year.eq(year)]
    nerdf = utils.standardize(utils.read_df_s3(f"{pub.name}/ner/ner_full_{year}.csv"), pub, drop_dups=False)
    nyr = (
        nerdf.drop_duplicates(subset=nerdf.columns.difference(['ner_index']))
        .loc[lambda d: d.label_.eq("PERSON")]
    )
    # ppl[ppl.Year.eq(year)]
    if len(qyr) != 0 and len(nyr) != 0:
        # qyr = qyr.head()
        qyr['entities'] = qyr.progress_apply(find_entity, ner=nyr, axis=1)
        x = qyr.entities.apply(pd.Series)
        for col in x.columns:
            for i, type_ in enumerate(("entity", "nerindex")):
                qyr['{}_{}'.format(type_, str(col+1))]=[k[i] if isinstance(k,list) else k for k in x[col]]
        # convert to ints
        for col in qyr.filter(regex="nerindex").columns:
            if qyr[col].dtype == "float64":
                qyr[col] = qyr[col].astype(pd.Int64Dtype())#)
        qdfls.append(qyr)
    else:
        print(f"only found {len(qyr)} quotes and {len(nyr)} entities")
    # run for 2011-2020; then try 2021 individually
    utils.df_to_s3(qyr, f"{pub.name}/quotes/q_{year}_full_edits_new716.csv")
    
# qdfconcat = pd.concat(qdfls)

100%|██████████| 88250/88250 [02:10<00:00, 675.11it/s]
/tmp/ipykernel_25749/732475428.py:31: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  x = qyr.entities.apply(pd.Series)
100%|██████████| 92125/92125 [02:15<00:00, 677.48it/s]
/tmp/ipykernel_25749/732475428.py:31: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  x = qyr.entities.apply(pd.Series)
100%|██████████| 157240/157240 [03:57<00:00, 662.97it/s]
/tmp/ipykernel_25749/732475428.py:31: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  x = qyr.entities.apply(pd.Series)
100%|██████████| 144723/144723 [03:35<00:00, 673.02it/s]
/tmp/ipykernel_25749/732475428.py:31: FutureWarning: The default dtype 

In [1]:
qdfconcat=concat2s3(qdfconcat, pub)

NameError: name 'concat2s3' is not defined

In [17]:
qdfls = []
for year in range(2011, 2022):
    # year = 2019
    # make it a manageable size
    qyr = quotedf[quotedf.Year.eq(year)]
    nyr = ppl[ppl.Year.eq(year)]
    if len(qyr) != 0 and len(nyr) != 0:
        qyr['entities'] = qyr.progress_apply(find_entity, ner=nyr, axis=1)
        x = qyr.entities.apply(pd.Series)
        for col in x.columns:
            for i, type_ in enumerate(("entity", "nerindex")):
                qyr['{}_{}'.format(type_, str(col+1))]=[k[i] if isinstance(k,list) else k for k in x[col]]
        # convert to ints
        for col in qyr.filter(regex="nerindex").columns:
            if qyr[col].dtype == "float64":
                qyr[col] = qyr[col].astype(pd.Int64Dtype())#)
        qdfls.append(qyr)
    else:
        print(f"only found {len(qyr)} quotes and {len(nyr)} entities")

qdfconcat = pd.concat(qdfls)

only found 0 quotes and 0 entities


100%|██████████| 1/1 [00:00<00:00, 108.33it/s]
/tmp/ipykernel_14389/2014587900.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qyr['entities'] = qyr.progress_apply(find_entity, ner=nyr, axis=1)
/tmp/ipykernel_14389/2014587900.py:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  x= qyr.entities.apply(pd.Series)


only found 0 quotes and 0 entities


100%|██████████| 130/130 [00:00<00:00, 394.42it/s]
/tmp/ipykernel_14389/2014587900.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qyr['entities'] = qyr.progress_apply(find_entity, ner=nyr, axis=1)
/tmp/ipykernel_14389/2014587900.py:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  x= qyr.entities.apply(pd.Series)
/tmp/ipykernel_14389/2014587900.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [22]:
qdfconcat.drop(['entity_2', 'entity_3', 'entity_4',
                'entity_5', 'entity_6', 'nerindex_2', 
                'nerindex_3','nerindex_4','nerindex_5',
                'nerindex_6', "Unnamed: 0.1", "Unnamed: 0", "publication"], 
               axis=1)

,speaker,speaker_index,quote,quote_index,verb,verb_index,quote_token_count,quote_type,is_floating_quote,Art_id,prepro_speaker,single_speaker,long_speaker,Publication,Year,entities,entity_1,nerindex_1
262,peaceful elections,"(1311,1329)",", most agencies are due to pull out by 2013 le...","(1348,1448)",concluded,"(1339,1348)",20,SVC,False,post-13763,peaceful elections,False,peaceful elections,hkfp,2012,[],NaN,<NA>
27,he,"(2491,2493)",“They want to help but don’t want to show thei...,"(2435,2488)",says,"(2494,2498)",13,QCQSV,False,post-13566,he,True,NaN,hkfp,2014,[],NaN,<NA>
28,Cheung,"(2994,3000)",it will be gathered and placed in the nearest ...,"(3006,3063)",says,"(3001,3005)",11,SVC,False,post-13566,cheung,True,bosco cheung,hkfp,2014,[],NaN,<NA>
29,"\nStudent volunteer, Debbie Shek, 23, who help...","(3204,3278)",there have been few problems with the system,"(3284,3328)",says,"(3279,3283)",8,SVC,False,post-13566,\nstudent volunteer debbie shek 23 who helps o...,False,\nstudent volunteer debbie shek 23 who helps o...,hkfp,2014,"[[Debbie Shek, 157]]",Debbie Shek,157
30,she,"(3535,3538)","They were confronted by protesters, then “drop...","(3451,3532)",says,"(3539,3543)",14,CSV,False,post-13566,she,True,NaN,hkfp,2014,[],NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96393,the ones we met along the way,"(561,590)",", they do the hike every morning and they are ...","(590,651)",according to,"(548,560)",14,AccordingTo,False,post-339012,the ones we met along the way,False,the ones we met along the way,hkfp,2021,[],NaN,<NA>
96394,\nA little further up the path,"(2372,2401)",that you are on the right track,"(2454,2485)",indicating,"(2443,2453)",7,SVC,False,post-339012,\na little further up the path,False,\na little further up the path,hkfp,2021,[],NaN,<NA>
96437,the bill passed in May,"(989,1011)",", while the total number of seats in LegCo inc...","(1011,1154)",according to,"(976,988)",28,AccordingTo,False,post-352679,the bill passed in may,False,the bill passed in may,hkfp,2021,[],NaN,<NA>
96438,she,"(4524,4527)",\n“It is a really tough decision to delay the ...,"(4347,4521)",said,"(4528,4532)",37,CSV,False,post-352679,she,True,NaN,hkfp,2021,[],NaN,<NA>
